In [ ]:
import time

from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from awkward import JaggedArray
import numpy as np


In [ ]:
# Look at ProcessorABC to see the expected methods and what they are supposed to do
class TTGammaProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        mass_axis = hist.Bin("mass", r"$m_{\mu\mu}$ [GeV]", 600, 0.25, 300)
        pt_axis = hist.Bin("pt", r"$p_{T,\gamma}$ [GeV]", 300, 0., 300)
        mult_axis = hist.Bin("N", r"Multiplicity", 10, 0, 10)

        self._accumulator = processor.dict_accumulator({
#             'mass': hist.Hist("Counts", dataset_axis, mass_axis),
#             'mass_near': hist.Hist("Counts", dataset_axis, mass_axis),
#             'mass_far': hist.Hist("Counts", dataset_axis, mass_axis),
            'pt_gamma': hist.Hist("Counts", dataset_axis, pt_axis),
            'pt_jet_pho': hist.Hist("Counts", dataset_axis, pt_axis),
            'pt_jet_pre': hist.Hist("Counts", dataset_axis, pt_axis),
            'jet_mult_pho': hist.Hist("Counts", dataset_axis, mult_axis),
            'jet_mult_pre': hist.Hist("Counts", dataset_axis, mult_axis),
#            'eventList' : processor.column_accumulator(np.array([],dtype=np.int64)),
            'cutflow': processor.defaultdict_accumulator(int),
#             'cutflow2': processor.defaultdict_accumulator( processor.defaultdict_accumulator(int) ),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        output = self.accumulator.identity()

        dataset = df['dataset']

        if '2016' in dataset:
            year=2016
            muonTrigger = df['HLT_IsoMu24'] | df['HLT_IsoTkMu24']
            eleTrigger = df['HLT_Ele32_eta2p1_WPTight_Gsf']
            photonBitMapName = 'Photon_cutBased'
        elif '2017' in dataset:
            year=2017
            muonTrigger = df['HLT_IsoMu27']
            eleTrigger = df['HLT_Ele32_WPTight_Gsf']
            photonBitMapName = 'Photon_cutBasedBitmap'
        elif '2018' in dataset:
            year=2018
            muonTrigger = df['HLT_IsoMu27']
            eleTrigger = df['HLT_Ele35_WPTight_Gsf']
            photonBitMapName = 'Photon_cutBasedBitmap'

        presel = muonTrigger
        
#         singleEvent = (df['run']==26) & (df['event']==925)
#         presel = presel & singleEvent
        
        output['cutflow']['all events'] += presel.size
        output['cutflow']['trigger'] += presel.sum()
        
        filters = (df['Flag_goodVertices'] &
                   df['Flag_globalSuperTightHalo2016Filter'] &
                   df['Flag_HBHENoiseFilter'] &
                   df['Flag_HBHENoiseIsoFilter'] &
                   df['Flag_EcalDeadCellTriggerPrimitiveFilter'] &
                   df['Flag_BadPFMuonFilter']
                  )
        
        presel = presel & filters
        output['cutflow']['filter'] += presel.sum()        
        
        muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'],
            eta=df['Muon_eta'],
            phi=df['Muon_phi'],
            mass=df['Muon_mass'],
            charge=df['Muon_charge'],
            relIso=df['Muon_pfRelIso03_all'],
            tightId=df['Muon_tightId'],
            isPFcand=df['Muon_isPFcand'],
            isTracker=df['Muon_isTracker'],
            isGlobal=df['Muon_isGlobal'],           
            )
        
        electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'],
            eta=df['Electron_eta'],
            phi=df['Electron_phi'],
            mass=df['Electron_mass'],
            charge=df['Electron_charge'],
            cutBased=df['Electron_cutBased'],
            d0=df['Electron_dxy'],
            dz=df['Electron_dz'],
        )

        jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['Jet_pt'],
            eta=df['Jet_eta'],
            phi=df['Jet_phi'],
            mass=df['Jet_mass'],
            jetId=df['Jet_jetId'],
            btag=df['Jet_btagDeepB'],
            hadFlav=df['Jet_hadronFlavour'],
            genIdx=df['Jet_genJetIdx'],
        )

        photons = JaggedCandidateArray.candidatesfromcounts(
            df['nPhoton'],
            pt=df['Photon_pt'],
            eta=df['Photon_eta'],
            phi=df['Photon_phi'],
            mass=df['Photon_mass'],
            photonId=df[photonBitMapName],
            passEleVeto=df['Photon_electronVeto'],
            pixelSeed=df['Photon_pixelSeed'],
        )
            

        muonSelectTight = ((muons.pt>30) & 
                           (abs(muons.eta)<2.4) & 
                           (muons.tightId) & 
                           (muons.relIso < 0.15)
                          )
        
        muonSelectLoose = ((muons.pt>15) & 
                           (abs(muons.eta)<2.4) & 
                           ((muons.isPFcand) & (muons.isTracker | muons.isGlobal)) & 
                           (muons.relIso < 0.15) &
                           np.invert(muonSelectTight)
                          )

        eleEtaGap = (abs(electrons.eta) < 1.4442) | (abs(electrons.eta) > 1.566)
        elePassD0 = ((abs(electrons.eta) < 1.4442) & (abs(electrons.d0) < 0.05) |
                     (abs(electrons.eta) < 1.566)  & (abs(electrons.d0) < 0.1)
                    )
        elePassDZ = ((abs(electrons.eta) < 1.4442) & (abs(electrons.dz) < 0.1) |
                     (abs(electrons.eta) < 1.566)  & (abs(electrons.dz) < 0.2)
                    )

        electronSelectTight = ((electrons.pt>35) & 
                               (abs(electrons.eta)<2.1) & 
                               eleEtaGap &      
                               (electrons.cutBased>=4) &
                               elePassD0 & 
                               elePassDZ
                              )

        electronSelectLoose = ((electrons.pt>15) & 
                               (abs(electrons.eta)<2.5) & 
                               eleEtaGap &      
                               (electrons.cutBased>=1) &
                               elePassD0 & 
                               elePassDZ & 
                               np.invert(electronSelectTight)
                              )
#        muonSelectLoose = (muons.pt>30) & (abs(muons.eta)<2.4) & (muons.tightId) & (muons.relIso < 0.15)
#        muonSelectLoose = (muons.pt>15) & (abs(muons.eta)<2.4) & (muons.tightId) & (muons.relIso < 0.25) & np.invert(muonSelectTight)
        #looseMuons = muons[muonSelectLoose]
        
        tightMuon = muons[muonSelectTight]
        looseMuon = muons[muonSelectLoose]

        tightElectron = electrons[electronSelectTight]
        looseElectron = electrons[electronSelectLoose]

#         output['cutflow']['hasMuons'] += muonSelectTight.any().sum()
        
        presel = (tightMuon.counts == 1) & presel
        output['cutflow']['oneMuon'] += presel.sum()

        presel = (tightElectron.counts == 0) & presel
        output['cutflow']['eleVeto'] += presel.sum()

        presel = (looseMuon.counts == 0) & presel
        output['cutflow']['noLooseMuons'] += presel.sum()

        presel = (looseElectron.counts == 0) & presel
        output['cutflow']['noLooseElectrons'] += presel.sum()
        

        
        #### Calculate deltaR between photon and nearest muon
        ####### make combination pairs
        phoMu = photons['p4'].cross(tightMuon['p4'],nested=True)
        ####### check delta R of each combination, if min is >0.1 it is okay, or if there are no tight muons it passes
        dRmu = (phoMu.i0.delta_r(phoMu.i1)>0.1).all() | (tightMuon.counts==0)

        phoEle = photons['p4'].cross(tightElectron['p4'],nested=True)
        dRele = ((phoEle.i0.delta_r(phoEle.i1)).min()>0.1) | (tightElectron.counts==0)
        
        photonSelect = ((photons.pt>20) & 
                        (abs(photons.eta) < 1.4442) &
                        ((photons.photonId >> 1 & 1) == 1) & 
                        (photons.passEleVeto) & 
                        np.invert(photons.pixelSeed) & 
                        dRmu & dRele
                       )
        tightPhotons = photons[photonSelect]
        
        jetIDbit = 1
        if year>2016: jetIDbit=2

        jetMu = jets['p4'].cross(tightMuon['p4'],nested=True)
        dRmu = ((jetMu.i0.delta_r(jetMu.i1)).min()>0.4) | (tightMuon.counts==0)

        jetEle = jets['p4'].cross(tightElectron['p4'],nested=True)
        dRele = ((jetEle.i0.delta_r(jetEle.i1)).min()>0.4) | (tightElectron.counts==0)

        jetPho = jets['p4'].cross(tightPhotons['p4'],nested=True)
        dRpho = ((jetPho.i0.delta_r(jetPho.i1)).min()>0.1) | (tightPhotons.counts==0)
        
        jetSelect = ((jets.pt > 30) &
                     (abs(jets.eta) < 2.4) &
                     ((jets.jetId >> jetIDbit & 1)==1) &
                     dRmu & dRele & dRpho                    
                    )

        tightJets = jets[jetSelect]
        
        bTagWP = 0.6321
        if year == 2017:
            bTagWP = 0.4941
        if year == 2018:
            bTagWP = 0.4184
        bJets = jets[jetSelect & (jets.btag>bTagWP)]
        
        presel = (tightJets.counts >= 2) & presel
        output['cutflow']['twoJets'] += presel.sum()

        presel = (tightJets.counts >= 4) & presel
        output['cutflow']['fourJets'] += presel.sum()
        
        presel = (bJets.counts >= 1) & presel
        output['cutflow']['bTag'] += presel.sum()
        
        phosel = (tightPhotons.counts >= 1) & presel
        output['cutflow']['phosel'] += phosel.sum()
        
#         print(tightPhotons)
#         print(tightPhotons.p4)
#         print(tightPhotons.p4.pt[:,:1])
#         print(tightPhotons.p4.pt[:,:1][phosel])
#         print(tightPhotons.p4.pt[:,:1][phosel].flatten())
                
        output['pt_gamma'].fill(dataset=dataset,
                            pt=tightPhotons.p4.pt[:,:1][phosel].flatten())
        output['pt_jet_pho'].fill(dataset=dataset,
                            pt=tightJets.p4.pt[:,:1][phosel].flatten())
        output['pt_jet_pre'].fill(dataset=dataset,
                            pt=tightJets.p4.pt[:,:1][presel].flatten())
        output['jet_mult_pho'].fill(dataset=dataset,
                                N=tightJets[phosel].counts.flatten())
        output['jet_mult_pre'].fill(dataset=dataset,
                                N=tightJets[presel].counts.flatten())
        
        output['eventList'].add(df['event'][phosel].flatten())
        
        
        
        return output

    def postprocess(self, accumulator):
        return accumulator


In [ ]:
tstart = time.time()

fileset = {
#     'TTGamma2016_NoFullyHad': [
#         'Files/myNanoProdMc2016_job0.root',
#         'Files/myNanoProdMc2016_job1.root',
#         'Files/myNanoProdMc2016_job2.root',
#         'Files/myNanoProdMc2016_job3.root',
#         'Files/myNanoProdMc2016_job4.root',
#         'Files/myNanoProdMc2016_job5.root',
#         'Files/myNanoProdMc2016_job6.root',
#         'Files/myNanoProdMc2016_job7.root',
#         'Files/myNanoProdMc2016_job8.root',
#         'Files/myNanoProdMc2016_job9.root',
#     ],
    'TTGamma2017_NoFullyHad': [
        'Files/myNanoProdMc2017_job0.root',
        'Files/myNanoProdMc2017_job1.root',
        'Files/myNanoProdMc2017_job2.root',
        'Files/myNanoProdMc2017_job3.root',
        'Files/myNanoProdMc2017_job4.root',
        'Files/myNanoProdMc2017_job5.root',
        'Files/myNanoProdMc2017_job6.root',
        'Files/myNanoProdMc2017_job7.root',
        'Files/myNanoProdMc2017_job8.root',
        'Files/myNanoProdMc2017_job9.root',
    ],
#     'TTGamma2018_NoFullyHad': [
#         'Files/myNanoProdMc2018_job0.root',
#         'Files/myNanoProdMc2018_job1.root',
#         'Files/myNanoProdMc2018_job2.root',
#         'Files/myNanoProdMc2018_job3.root',
#         'Files/myNanoProdMc2018_job4.root',
#         'Files/myNanoProdMc2018_job5.root',
#         'Files/myNanoProdMc2018_job6.root',
#         'Files/myNanoProdMc2018_job7.root',
#         'Files/myNanoProdMc2018_job8.root',
#         'Files/myNanoProdMc2018_job9.root',
#     ],
}

output = processor.run_uproot_job(fileset,
                                  treename='Events',
                                  processor_instance=TTGammaProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers': 6, 'flatten': True},
                                  chunksize=10000,
                                 )

elapsed = time.time() - tstart
print(output)
print(elapsed)

In [ ]:
fig, ax, _ = hist.plot1d(output['pt_jet_pho'], overlay='dataset')
ax.set_xlim(0,250)


In [ ]:
import uproot

In [ ]:
_tree = uproot.open('/uscms_data/d3/npoudyal/TTGammaSemiLeptonic13TeV/CMSSW_10_2_14/src/NanoAOD_TTGamma_13TeV/TTGamma_noFullyHad_2017_AnalysisNtuple.root')['AnalysisTree']

In [ ]:
runs = _tree.array('run')
event = _tree.array('event')
muSel = (_tree.array('nMu')==1)
jetSel = (_tree.array('nJet')>=4) & muSel
bjetSel = (_tree.array('nBJet')>=1) & jetSel
phoSel = (_tree.array('nPho')>=1) & bjetSel

In [ ]:
print(muSel.sum())
print(jetSel.sum())
print(bjetSel.sum())
print(phoSel.sum())


In [ ]:
sel = jetSel
np.array([runs[sel],event[sel]]).transpose()